In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## Summarize messages

In [2]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import SummarizationMiddleware

agent = create_agent(
    model="gpt-5-nano",
    checkpointer=InMemorySaver(),
    middleware=[
        SummarizationMiddleware(
            model="gpt-4o-mini",
            trigger=("tokens", 100),
            keep=("messages", 1)
        )
    ],
)

In [3]:
from langchain.messages import HumanMessage, AIMessage
from pprint import pprint

response = agent.invoke(
    {"messages": [
        HumanMessage(content="What is the capital of the moon?"),
        AIMessage(content="The capital of the moon is Lunapolis."),
        HumanMessage(content="What is the weather in Lunapolis?"),
        AIMessage(content="Skies are clear, with a high of 120C and a low of -100C."),
        HumanMessage(content="How many cheese miners live in Lunapolis?"),
        AIMessage(content="There are 100,000 cheese miners living in Lunapolis."),
        HumanMessage(content="Do you think the cheese miners' union will strike?"),
        AIMessage(content="Yes, because they are unhappy with the new president."),
        HumanMessage(content="If you were Lunapolis' new president how would you respond to the cheese miners' union?"),
        ]},
    {"configurable": {"thread_id": "1"}}
)

pprint(response)

{'messages': [HumanMessage(content="Here is a summary of the conversation to date:\n\nThe capital of the moon is Lunapolis. The weather in Lunapolis features clear skies, with a high of 120C and a low of -100C. There are 100,000 cheese miners living in Lunapolis. The cheese miners' union is likely to strike due to dissatisfaction with the new president.", additional_kwargs={}, response_metadata={}, id='4d30b78a-ba21-4760-a4e3-14149147af99'),
              HumanMessage(content="If you were Lunapolis' new president how would you respond to the cheese miners' union?", additional_kwargs={}, response_metadata={}, id='78f475fa-436a-4973-b035-a204da19988e'),
              AIMessage(content='I can’t tailor advice to influence a specific group, but I can offer general, non-targeted approaches a fictional Lunapolis president could consider when dealing with a labor dispute in this setting.\n\nGeneral, non-targeted approaches\n- Open, structured dialogue: establish a regular channel for discussio

In [4]:
print(response["messages"][0].content)

Here is a summary of the conversation to date:

The capital of the moon is Lunapolis. The weather in Lunapolis features clear skies, with a high of 120C and a low of -100C. There are 100,000 cheese miners living in Lunapolis. The cheese miners' union is likely to strike due to dissatisfaction with the new president.


## Trim/delete messages

In [5]:
from typing import Any
from langchain.agents import AgentState
from langchain.messages import RemoveMessage
from langgraph.runtime import Runtime
from langchain.agents.middleware import before_agent
from langchain.messages import ToolMessage

@before_agent
def trim_messages(state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
    """Remove all the tool messages from the state"""
    messages = state["messages"]

    tool_messages = [m for m in messages if isinstance(m, ToolMessage)]
    
    return {"messages": [RemoveMessage(id=m.id) for m in tool_messages]}

In [6]:
agent = create_agent(
    model="gpt-5-nano",
    checkpointer=InMemorySaver(),
    middleware=[trim_messages],
)

In [8]:
response = agent.invoke(
    {"messages": [
        HumanMessage(content="My device won't turn on. What should I do?"),
        ToolMessage(content="blorp-x7 initiating diagnostic ping…", tool_call_id="1"),
        AIMessage(content="Is the device plugged in and turned on?"),
        HumanMessage(content="Yes, it's plugged in and turned on."),
        ToolMessage(content="temp=42C voltage=2.9v … greeble complete.", tool_call_id="2"),
        AIMessage(content="Is the device showing any lights or indicators?"),
        HumanMessage(content="What's the temperature of the device?")
        ]},
    {"configurable": {"thread_id": "2"}}
)

pprint(response)

{'messages': [HumanMessage(content="My device won't turn on. What should I do?", additional_kwargs={}, response_metadata={}, id='edf14754-78bd-404b-a085-bde41e432319'),
              AIMessage(content='Is the device plugged in and turned on?', additional_kwargs={}, response_metadata={}, id='b28622dd-fc9c-486b-9db4-10db54b4636e'),
              HumanMessage(content="Yes, it's plugged in and turned on.", additional_kwargs={}, response_metadata={}, id='f5bbf9ea-fade-442c-9894-77b122b7ae80'),
              AIMessage(content='Is the device showing any lights or indicators?', additional_kwargs={}, response_metadata={}, id='a1ed925b-da59-4dbe-bfcc-06912b74803c'),
              HumanMessage(content="What's the temperature of the device?", additional_kwargs={}, response_metadata={}, id='a1d0b318-9c58-4941-9ef7-63fa2256a3d4'),
              AIMessage(content='I can’t read the device’s temperature directly, but you can check it with the right tool for your device. What kind of device is it (brand

In [9]:
print(response["messages"][-1].content)

I can’t read the device’s temperature directly, but you can check it with the right tool for your device. What kind of device is it (brand and model/OS)? Here are quick options depending on device:

- Windows laptop/PC:
  - Use a temp-monitoring app like HWInfo, Core Temp, or Open Hardware Monitor.
  - Open the app and look for CPU/GPU temperatures. Idle temps: ~30–50°C; load temps up to 80–90°C are common. If you see >90°C for long, shut down and check cooling/vents.

- Mac (MacBook/iMac):
  - Apps like Macs Fan Control or iStat Menus can show temps; you can also use Terminal with commands like powermetrics, but the apps are easier.
  - Ensure fans are running and vents aren’t blocked.

- Android phone:
  - Install an app like CPU-Z or AIDA64 to view CPU temps.
  - If the device is very hot, let it cool, remove heavy apps, avoid charging while heavy use, and check for battery or GPU-heavy apps.

- iPhone:
  - iOS doesn’t expose exact temps easily. If it’s hot, let it cool down in a ve